In [1]:
import numpy as np
import pandas as pd

# Input file parameter

First we must indicated how many set are there in the TableA

In [2]:
set_total = 0

# Train file create

Open the DCADE output TableA.txt and read page file to give each node data corresponding labels (column number).

In [11]:
table_name = "./data/TableA.txt"
print("Table Opening:" + table_name + "\n")
table_a = open(table_name, "r")
output_name = "./data/train_raw.csv"
print("Generating:" + output_name + "\n")

Set_index = {}

output = open(output_name, "w")
output.write("Leafnode\tPTypeSet\tTypeSet\tContentid\tPathid\tSimseqid\tPath\tContent\tLabel\n")

line = table_a.readline()
slot = line.rstrip("\n").split("\t") # Read Table line in line and split by \t saved in slot.

while(slot[0]!="ColType"):
    '''
    Loop until we find the "ColType" in the table file to start generate training data.
    '''
    print("find coltype")
    line = table_a.readline()
    slot = line.rstrip("\n").split("\t")
col = []
line = table_a.readline() # Read first line
while(line != ""):
    slot = line.rstrip("\n").split("\t")
    page_name = slot[0]
    node_col = slot[1:] # Save each column's data in node_col
    col = []
    for i in node_col: # Split Same column data and save in list col
        tmp = i.split(" ")
        col.append(tmp)
    
    current_page = 0
    for pos in range(len(col)): # Loop all col num
        if len(col[pos]) == 1 and not col[pos][0].isnumeric(): # Find "X-X" like sets
            #print("Find : "+col[pos][0])
            current_page = col[pos][0].split("-")[1]
            current_page = int(current_page)
            set_num = col[pos][0].split("-")[0]
            set_num = int(set_num)
            if set_num not in Set_index.keys():
                #print("Add new set in dict col num:"+str(pos+1))
                Set_index[set_num] = pos+1 # Record it in dict
    
    page_file = pd.read_csv("./Output/"+page_name, sep='\t') # Read page file from DECADE Output file.
    
    col_name = []
    for i in page_file.columns: # Save all column name
        col_name.append(i)
    
    print("open "+ page_name)
    label_dict = {} # Define Dict of labels for each node
    for leafnode in page_file[col_name[0]]: # Loop each node in page
        for pos in range(len(col)): # Loop all col
            if str(leafnode) in col[pos]: # if node is in this column
                if leafnode not in label_dict.keys(): # and if this node not in dict save it
                    label_dict[leafnode] = pos+1 # Shift one column for every node (zero is for empty).
                #print(str(leafnode) + "\'s label: " + str(pos))
                break
    
    for set_count in range(set_total): # Loop every set and give each node a label, which is the column number of the set.
        with open("./data/Set-"+str(set_count+1)+".txt", "r") as set_file:
            line = set_file.readline()
            slot = line.rstrip("\n").split("\t")
            
            while(slot[0] != "ColType"):
                line = set_file.readline()
                slot = line.rstrip("\n").split("\t")
            
            line = set_file.readline()
            slot = line.rstrip("\n").split("\t")
            while(line!=""):
                file_set_info = slot[0].split("-")
                file_set_num = set_count
                file_set_page = int(file_set_info[1])
                for node in slot[1:]:
                    if not node.isnumeric():
                        continue
                    node = int(node)
                    if file_set_page == current_page:
                        label_dict[node] = Set_index[set_count+1]
                line = set_file.readline()
                slot = line.rstrip("\n").split("\t")
    error = False
    '''for i in range(max(label_dict.keys())+1):
        if i not in label_dict:
            error = True
            print(i)'''
    #print(label_dict)
    #count = 0
    for i in range(max(label_dict.keys())+1): # Loop each recorded node and output as train file
        Leafnode = page_file[col_name[0]][i]
        PTypeSetid = page_file[col_name[6]][i]
        TypeSetid = page_file[col_name[7]][i]
        Contentid = page_file[col_name[8]][i].split("-")[1]
        Pathid = page_file[col_name[9]][i]
        SimSeqid = page_file[col_name[10]][i]
        Path = page_file[col_name[2]][i]
        Content = page_file[col_name[1]][i]
        output.write(str(Leafnode) + "\t")
        output.write(str(PTypeSetid) + "\t")
        output.write(str(TypeSetid) + "\t")
        output.write(str(Contentid) + "\t")
        output.write(str(Pathid) + "\t")
        output.write(str(SimSeqid) + "\t")
        output.write(str(Path) + "\t")
        output.write(str(Content) + "\t")
        if i not in label_dict:
            output.write("0\n")
        else:
            output.write(str(label_dict[i]) + "\n")
    line = table_a.readline()
output.close()

Table Opening:./data/TableA.txt

Generating:./data/train_raw.csv

open Webpage-0.txt
open Webpage-1015.txt
open Webpage-1026.txt
open Webpage-133.txt
open Webpage-197.txt
open Webpage-200.txt
open Webpage-22.txt
open Webpage-225.txt
open Webpage-24.txt
open Webpage-319.txt
open Webpage-322.txt
open Webpage-325.txt
open Webpage-326.txt
open Webpage-327.txt
open Webpage-329.txt
open Webpage-330.txt
open Webpage-331.txt
open Webpage-332.txt
open Webpage-333.txt
open Webpage-334.txt
open Webpage-335.txt
open Webpage-336.txt
open Webpage-337.txt
open Webpage-338.txt
open Webpage-342.txt
open Webpage-343.txt
open Webpage-344.txt
open Webpage-346.txt
open Webpage-347.txt
open Webpage-349.txt
open Webpage-350.txt
open Webpage-398.txt
open Webpage-419.txt
open Webpage-430.txt
open Webpage-459.txt
open Webpage-466.txt
open Webpage-493.txt
open Webpage-5.txt
open Webpage-506.txt
open Webpage-527.txt
open Webpage-539.txt
open Webpage-580.txt
open Webpage-608.txt
open Webpage-671.txt
open Webpage-6

In [12]:
if not error:
    print("Done")
else:
    print("Error")

Done


# Record Set data

Recorded the node index that is in a set and saved it in file.

In [5]:
if set_total > 0:
    with open("./data/Set_idx.txt", "w") as set_file:
        set_file.write(str(Set_index))
    print(Set_index)

# GA file create

GA file is for testing.

In [6]:
table_name = "./data/GA.txt"
print("Table Opening:" + table_name + "\n")
table_a = open(table_name, "r")
output_name = "./data/ytest_raw.csv"
print("Generating:" + output_name + "\n")

Table Opening:./data/GA.txt

Generating:./data/ytest_raw.csv



Testing file generation is same as training file generation, but give all the nodes label 0.

In [7]:
Set_index = {}
output = open(output_name, "w")
output.write("Leafnode\tPTypeSet\tTypeSet\tContentid\tPathid\tSimseqid\tPath\tContent\tLabel\n")
line = table_a.readline()
slot = line.rstrip("\n").split("\t")
while(slot[0]!="ColType"):
    print("find coltype")
    line = table_a.readline()
    slot = line.rstrip("\n").split("\t")
line = table_a.readline() # Read
while(line != ""):
    slot = line.rstrip("\n").split("\t")
    page_name = slot[0]
    if slot[0]=="":
        break
    page_file = pd.read_csv("./Output/"+page_name, sep='\t')
    col_name = []
    for i in page_file.columns:
        col_name.append(i)
    print("open "+ page_name)
    label_dict = {}
    for node in range(len(page_file[col_name[0]])):
        Leafnode = page_file[col_name[0]][node]
        PTypeSetid = page_file[col_name[6]][node]
        TypeSetid = page_file[col_name[7]][node]
        Contentid = page_file[col_name[8]][node].split("-")[1]
        Pathid = page_file[col_name[9]][node]
        SimSeqid = page_file[col_name[10]][node]
        Path = page_file[col_name[2]][node]
        Content = page_file[col_name[1]][node]
        output.write(str(Leafnode) + "\t")
        output.write(str(PTypeSetid) + "\t")
        output.write(str(TypeSetid) + "\t")
        output.write(str(Contentid) + "\t")
        output.write(str(Pathid) + "\t")
        output.write(str(SimSeqid) + "\t")
        output.write(str(Path) + "\t")
        output.write(str(Content) + "\t")
        output.write(str(0) + "\n")
    line = table_a.readline() # Read
output.close()

open Webpage-0.txt
open Webpage-1.txt
open Webpage-2.txt
open Webpage-3.txt
open Webpage-4.txt
open Webpage-5.txt
open Webpage-6.txt
open Webpage-7.txt
open Webpage-8.txt
open Webpage-9.txt
open Webpage-10.txt
open Webpage-11.txt
open Webpage-12.txt
open Webpage-13.txt
open Webpage-14.txt
open Webpage-15.txt
open Webpage-16.txt
open Webpage-17.txt
open Webpage-18.txt
open Webpage-19.txt
open Webpage-20.txt
open Webpage-21.txt
open Webpage-22.txt
open Webpage-23.txt
open Webpage-24.txt
open Webpage-25.txt
open Webpage-26.txt
open Webpage-27.txt
open Webpage-28.txt
open Webpage-29.txt
open Webpage-30.txt
open Webpage-31.txt
open Webpage-32.txt
open Webpage-33.txt
open Webpage-34.txt
open Webpage-35.txt
open Webpage-36.txt
open Webpage-37.txt
open Webpage-38.txt
open Webpage-39.txt
open Webpage-40.txt
open Webpage-41.txt
open Webpage-42.txt
open Webpage-43.txt
open Webpage-44.txt
open Webpage-45.txt
open Webpage-46.txt
open Webpage-47.txt
open Webpage-48.txt
open Webpage-49.txt
open Webpa

open Webpage-396.txt
open Webpage-397.txt
open Webpage-398.txt
open Webpage-399.txt
open Webpage-400.txt
open Webpage-401.txt
open Webpage-402.txt
open Webpage-403.txt
open Webpage-404.txt
open Webpage-405.txt
open Webpage-406.txt
open Webpage-407.txt
open Webpage-408.txt
open Webpage-409.txt
open Webpage-410.txt
open Webpage-411.txt
open Webpage-412.txt
open Webpage-413.txt
open Webpage-414.txt
open Webpage-415.txt
open Webpage-416.txt
open Webpage-417.txt
open Webpage-418.txt
open Webpage-419.txt
open Webpage-420.txt
open Webpage-421.txt
open Webpage-422.txt
open Webpage-423.txt
open Webpage-424.txt
open Webpage-425.txt
open Webpage-426.txt
open Webpage-427.txt
open Webpage-428.txt
open Webpage-429.txt
open Webpage-430.txt
open Webpage-431.txt
open Webpage-432.txt
open Webpage-433.txt
open Webpage-434.txt
open Webpage-435.txt
open Webpage-436.txt
open Webpage-437.txt
open Webpage-438.txt
open Webpage-439.txt
open Webpage-440.txt
open Webpage-441.txt
open Webpage-442.txt
open Webpage-

open Webpage-788.txt
open Webpage-789.txt
open Webpage-790.txt
open Webpage-791.txt
open Webpage-792.txt
open Webpage-793.txt
open Webpage-794.txt
open Webpage-795.txt
open Webpage-796.txt
open Webpage-797.txt
open Webpage-798.txt
open Webpage-799.txt
open Webpage-800.txt
open Webpage-801.txt
open Webpage-802.txt
open Webpage-803.txt
open Webpage-804.txt
open Webpage-805.txt
open Webpage-806.txt
open Webpage-807.txt
open Webpage-808.txt
open Webpage-809.txt
open Webpage-810.txt
open Webpage-811.txt
open Webpage-812.txt
open Webpage-813.txt
open Webpage-814.txt
open Webpage-815.txt
open Webpage-816.txt
open Webpage-817.txt
open Webpage-818.txt
open Webpage-819.txt
open Webpage-820.txt
open Webpage-821.txt
open Webpage-822.txt
open Webpage-823.txt
open Webpage-824.txt
open Webpage-825.txt
open Webpage-826.txt
open Webpage-827.txt
open Webpage-828.txt
open Webpage-829.txt
open Webpage-830.txt
open Webpage-831.txt
open Webpage-832.txt
open Webpage-833.txt
open Webpage-834.txt
open Webpage-

In [8]:
print("Done!")

Done!
